In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import Prepare_Data
import pyarrow 
from Prepare_Data import load_and_filter_market_returns
import data_run_files
#autoreload download #pip install ipython
%load_ext autoreload
%autoreload 2

#file_path_id_test="./data_test/top_5_percent_ids.csv"
# Sti til usa_dsf.parquet
file_path_usa_dsf = "./Data/usa_dsf.parquet"
file_path_usa = "./Data/usa.parquet"
file_path_market_returns = "./Data/market_returns.csv"
file_path_crsp_a_stock_with_id = "./Data/crsp_a_stock_codes.csv"
file_path_cluster_labels = "./Data/Cluster Labels.csv"
file_path_factor_details = "./Data/Factor Details.xlsx"
file_path_tsla = "./Data/tsla101.csv"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
#Input paths
rente_path = "Data/ff3_m.csv"

In [40]:
#output paths: 
risk_free_test_path = "./data_test/risk_free_thres_test.csv"
output_path_usa_dsf = "./data_test/usa_dsf_thres_test.parquet"
output_path_usa = "./data_test/usa_thres_test.parquet"
output_path_market_returns = "data_test/market_returns_thres_test.csv"

In [41]:
#Dates
start_date = pd.to_datetime('2004-12-31')
end_date = pd.to_datetime('2023-11-30')
#Threshold 
thres = 0.85 #(vi beholder de øverste 10%, hvis den er lig 0.9)

# Top_thres_ids.csv (csv fil med unikke id'er)

In [16]:
df_usa = pd.read_parquet(file_path_usa, engine='pyarrow')
print("Fil indlæst med succes.")

Fil indlæst med succes.


In [17]:
# Filtrer kun USA og id <= 99999
df_usa = df_usa[(df_usa["excntry"] == "USA") & (df_usa["id"] <= 99999)]

In [19]:
df_usa.loc[:, 'eom'] = pd.to_datetime(df_usa['eom'], errors='coerce')
# Filtrer kun data fra og med 2010
df_usa_filtered = df_usa[df_usa['eom'] >= start_date]

# Bekræft ændringen
print(df_usa_filtered.head())
print(f"Antal observationer efter filtrering: {len(df_usa_filtered)}")

# Valider at filtreringen virker
print(df_usa_filtered['eom'].min())  # Skal vise en dato >= start_date

        eom_ret     id                  eom  valid_ret excntry          me  \
692  2007-01-31  10026  2006-12-31 00:00:00       True     USA  766.521028   
693  2007-02-28  10026  2007-01-31 00:00:00       True     USA  765.372457   
694  2007-03-31  10026  2007-02-28 00:00:00       True     USA  735.892265   
695  2007-04-30  10026  2007-03-31 00:00:00       True     USA  733.131881   
696  2007-09-30  10026  2007-08-31 00:00:00       True     USA  701.246236   

      sic size_grp  ret_exc_lead1m    niq_su  ...     ni_be    ocf_at  \
692  2050    small       -0.007227  0.496813  ...  0.103420  0.154170   
693  2050    small       -0.042354  2.854474  ...  0.104773  0.161278   
694  2050    small       -0.007154  2.854474  ...  0.104773  0.161278   
695  2050    small       -0.017516  2.854474  ...  0.104773  0.161278   
696  2050    small       -0.073786  0.789103  ...  0.108646  0.169546   

     ocf_at_chg1  mispricing_perf  mispricing_mgmt       qmj  qmj_prof  \
692    -0.014051  

In [20]:
# Beregn median market equity for hver virksomhed
company_median_equity = df_usa_filtered.groupby('id')['market_equity'].median()

# Find 95. percentilen for virksomheders median market equity
threshold = company_median_equity.quantile(thres)

# Udvælg top 5% af virksomhederne (dem med median market equity >= threshold)
top_thres_ids = company_median_equity[company_median_equity >= threshold].index

# Filtrer df_usa_filtered, så det kun indeholder disse virksomheder
df_thres_ids = df_usa_filtered[df_usa_filtered['id'].isin(top_thres_ids)]

# Udskriv information
print(f"Grænseværdi for top 15% virksomheder: {threshold}")
print(f"Antal unikke virksomheder i top 15%: {len(top_thres_ids)}")

Grænseværdi for top 10% virksomheder: 6101.088460259999
Antal unikke virksomheder i top 10%: 872


In [21]:
top_thres_ids

Index([10078, 10104, 10107, 10108, 10138, 10145, 10147, 10225, 10299, 10324,
       ...
       93089, 93094, 93096, 93101, 93132, 93150, 93296, 93312, 93338, 93436],
      dtype='int32', name='id', length=872)

In [22]:
top_thres_ids.unique()

Index([10078, 10104, 10107, 10108, 10138, 10145, 10147, 10225, 10299, 10324,
       ...
       93089, 93094, 93096, 93101, 93132, 93150, 93296, 93312, 93338, 93436],
      dtype='int32', name='id', length=872)

In [28]:
# Find første og sidste dato i dataset
first_date = df_thres_ids['eom'].min()
last_date = df_thres_ids['eom'].max()

# Filtrer virksomheder, der har data på både første og sidste dato
df_thres_ids = df_thres_ids.groupby('id').filter(
    lambda x: (first_date in x['eom'].values) and (last_date in x['eom'].values)
)
# Udskriv antal unikke virksomheder efter filtrering
print(f"Antal unikke virksomheder med data på både første og sidste dato: {df_thres_ids['id'].nunique()}")

Antal unikke virksomheder med data på både første og sidste dato: 381


In [30]:
df_ids = pd.DataFrame(df_thres_ids['id'].unique(), columns=['id'])
# Gem til en CSV-fil uden index-kolonne
df_ids.to_csv("./data_test/top_thres_percent_ids.csv", index=False)

print("Fil gemt som 'top_thres_percent_ids.csv'")

Fil gemt som 'top_thres_percent_ids.csv'


# De andre filer (med antal id's fra thres og start-end date)

In [42]:
file_path_id_test = "./data_test/top_thres_percent_ids.csv"

In [46]:
risk_free = data_run_files.process_risk_free_rate(rente_path, start_date, end_date, output_path = risk_free_test_path)
# Filtrér ID'er for usa_dsf og usa
data_run_files.filter_ids_from_dataset(file_path_usa_dsf, file_path_id_test, output_path_usa_dsf, start_date)
data_run_files.filter_ids_from_dataset(file_path_usa, file_path_id_test, output_path_usa, start_date)
data_run_files.filter_and_save_data(file_path_market_returns, start_date, end_date, output_path_market_returns)
h_list = [1]  # Horisonter
dataret = data_run_files.monthly_returns(risk_free, h_list, output_path_usa)

Filtrering udført. Antal rækker efter filtrering: 228
Fil gemt som ./data_test/risk_free_thres_test.csv


ArrowMemoryError: malloc of size 8388608 failed